# Lab 6

This assignment can be divided into three parts, and each part contains some coding tasks:
1. Hidden Markov Model
    - Complete a model
    - Implement the **Forward Procedure**, **Backward Procedure**

Do **not** include any extra outputs in your final answer. If you use print() function for debugging, please run your code without them again before submitting.

## Q1: Hidden Markov Model

In this problem, you need to first complete a model by filling some missing values according to the existing ones, and then implement and run **Forward Procedure**, **Backward Procedure**, and the **Viterbi Algorithm**.

### Problem Setup

Consider an HMM with the following properties (unknown values are marked with "?").

States: $S = \{S_{1}, S_{2}, S_{3}, S_{4}\}$

Initial state probability distribution: $\pi = \{\pi_{1} = 0.3, \pi_{2} = 0.1, \pi_{3} = ?, \pi_{4} = 0.5\}$

Transition probability matrix A, where each entry $a_{ij}$ is the probability of moving from state $S_{i}$ to state $S_{j}$:

$$\begin{bmatrix}
? & 0.5 & 0.4 & 0\\
? & 0.3 & 0.4 & 0.1\\
? & 0.1 & 0.2 & 0.7 \\
? & 0.2 & 0.2 & 0.2
\end{bmatrix}$$

Symbols probability matrix $B$, where each entry $b_{jk} = b_{j}(k) = P[V_{k} \; | \; S_{j}]$ is the probability of yielding $V_{k}$ in state $S_{j}$:

$$\begin{bmatrix}
0.5 & 0.4 & ? \\
? & 0.3 & 0.1 \\
0.8 & ? & 0.1 \\
? & 0.3 & 0.7
\end{bmatrix}$$

Let $\lambda = (A, B, \pi)$.

Suppose we observe a sequence $O = V_{1}V_{3}V_{2}V_{1}$.

### Q1.1

Complete the model by filling the unknown values in the following code block and run the cell. Make sure the outputs are all **True**.



In [4]:
import numpy as np

M = 3 # vocabulary size
N = 4 # state number
V = [None, 0, 1, 2] # one-based, V[0] = None to align the indices
x = np.nan
# fill in the blank values for the following three arrays
pi = np.array([0.3, 0.1, 0.1, 0.5])
A = np.array([[1-0.5-0.4-0, 0.5, 0.4, 0], [1-0.3-0.4-0.1, 0.3, 0.4, 0.1], [1-0.1-0.2-0.7, 0.1, 0.2, 0.7], [1-0.2-0.2-0.2, 0.2, 0.2, 0.2]])
B = np.array([[0.5, 0.4, 1-0.5-0.4 ], [1-0.3-0.1, 0.3, 0.1], [0.8, 1-0.8-0.1 , 0.1], [1-0.3-0.7, 0.3, 0.7]])

O = [V[1], V[3], V[2], V[1]]

print("Sum of pi is one? " + str(np.linalg.norm(pi.sum() - 1) < 1e-9))
print("Sum of each row of A is one? " + str(np.linalg.norm(A.sum(axis = 1) - 1) < 1e-9))
print("Sum of each row of B is one? " + str(np.linalg.norm(B.sum(axis = 1) - 1) < 1e-9))

Sum of pi is one? True
Sum of each row of A is one? True
Sum of each row of B is one? True


### Q1.2

What is $P(O | \lambda)$? Answer the question by finishing implementing the **Forward Procedure** and the **Backward Procedure** below and run the cells.

***Hints***
- The two procedures should yield the same final answer.
- The forward procedure code prints the $\alpha$ values as a table while the backward procedure code prints the $\beta$ values as a table. Think of how the values are supposed to change over the steps/rows.

### **Forward Procedure**

In [5]:
T = len(O)
alpha = np.zeros((T, N))

# initialization
for i in range(N): # complete this loop
    ################### start of your code ###################
    alpha[0, i] = pi[i] * B[i, O[0]]
    #################### end of your code ####################

# inductive steps
for t in range(1, T): # complete this loop
    ################### start of your code ###################
    for j in range(N):
        for i in range(N):
            alpha[t, j] += alpha[t-1, i] * A[i, j]
        alpha[t, j] *= B[j, O[t]]
    #################### end of your code ####################
print("alpha values:")
print(alpha)
print()

# final answer
################### start of your code ###################
answer = np.sum(alpha[T-1, :]) # this variable should store your final answer
#################### end of your code ####################
print("P(O | lambda) = " + str(answer))

alpha values:
[[0.15       0.06       0.08       0.        ]
 [0.0027     0.0101     0.01       0.0434    ]
 [0.00786    0.004218   0.00158    0.005007  ]
 [0.0018162  0.00381288 0.00491888 0.        ]]

P(O | lambda) = 0.010547960000000002


### **Backward Procedure**

In [6]:
T = len(O)
beta = np.zeros((T, N))

# initialization
for i in range(N): # complete this loop
    ################### start of your code ###################
    beta[T-1, i] = 1
    #################### end of your code ###################

# inductive steps
for t in range(T - 2, -1, -1): # complete this loop
    ################### start of your code ###################
    for i in range(N):
        for j in range(N):
            beta[t, i] += A[i, j] * B[j, O[t+1]] * beta[t+1, j]
    #################### end of your code ###################
print("beta values:")
print(beta)
print()

# final answer
# complete this part
################### start of your code ###################
answer = np.sum(pi[:] * B[:, O[0]] * beta[0, :]) # this variable should store your final answer

#################### end of your code ####################
print("P(O | lambda) = " + str(answer))

beta values:
[[0.012724 0.023712 0.090208 0.0348  ]
 [0.1256   0.1308   0.1232   0.1764  ]
 [0.67     0.6      0.22     0.48    ]
 [1.       1.       1.       1.      ]]

P(O | lambda) = 0.010547960000000002


### Q1.3

Implement the **Viterbi Algorithm** to find the most likely sequence of hidden states that produced the observed sequence $O$.

In [7]:
# Implement the Viterbi Algorithm here

# initialization
delta = np.zeros((T, N))
psi = np.zeros((T, N), dtype=int)

# complete this loop
################### start of your code ###################
for i in range(N):
    delta[0, i] = pi[i] * B[i, O[0]]
    psi[0, i] = 0 # Or a placeholder indicating no previous state
#################### end of your code ####################

# inductive steps
for t in range(1, T): # complete this loop
    ################### start of your code ###################
    for j in range(N):
        max_prob = -1
        max_state = -1
        for i in range(N):
            prob = delta[t-1, i] * A[i, j]
            if prob > max_prob:
                max_prob = prob
                max_state = i
        delta[t, j] = max_prob * B[j, O[t]]
        psi[t, j] = max_state
    #################### end of your code ####################

# termination
################### start of your code ###################
best_path_prob = np.max(delta[T-1, :])
best_path_pointer = np.argmax(delta[T-1, :])
#################### end of your code ####################

# path backtracking
best_path = [0] * T
################### start of your code ###################
best_path[T-1] = best_path_pointer
for t in range(T-2, -1, -1):
    best_path[t] = psi[t+1, best_path[t+1]]
#################### end of your code ###################

print("Delta values:")
print(delta)
print()
print("Psi values:")
print(psi)
print()
print("Best path probability: " + str(best_path_prob))
print("Best path: " + str([s + 1 for s in best_path])) # Adding 1 to convert from 0-based index to 1-based state number

Delta values:
[[0.15       0.06       0.08       0.        ]
 [0.0015     0.0075     0.006      0.0392    ]
 [0.006272   0.002352   0.000784   0.002352  ]
 [0.0004704  0.0018816  0.00200704 0.        ]]

Psi values:
[[0 0 0 0]
 [0 0 0 2]
 [3 3 3 3]
 [3 0 0 2]]

Best path probability: 0.002007040000000001
Best path: [np.int64(3), np.int64(4), np.int64(1), np.int64(3)]
